In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
%cd /content/drive/MyDrive/Vehicle Crash Detection

/content/drive/MyDrive/Vehicle Crash Detection


In [2]:
%ls

 coco.txt   cr.mp4   VehicleCrash.ipynb  'Vehicle Dataset'/   yolov8s.pt


In [3]:

!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.3/709.3 kB 5.0 MB/s eta 0:00:00


In [8]:
%cd /content/drive/MyDrive/Vehicle Crash Detection/Vehicle Dataset

/content/drive/MyDrive/Vehicle Crash Detection/Vehicle Dataset


In [13]:


!yolo task=detect mode=train model=yolov8s.pt data= data.yaml epochs=25 imgsz=224 plots=True

Ultralytics YOLOv8.1.9 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=data.yaml, epochs=25, time=None, patience=50, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_w

In [ ]:
!yolo task=detect mode=predict model=yolov8n.pt source="images/1.jpg"

In [4]:
!pip install cvzone


  Preparing metadata (setup.py) ... done
  Created wheel for cvzone: filename=cvzone-1.6.1-py3-none-any.whl size=26297 sha256=4dd6ef8b216c4c5663c366bf14b67f8695b62418b3e909f325c91cfec2ab1b00
  Stored in directory: /root/.cache/pip/wheels/2c/9f/b3/92e945ac4a71bf727a92463f38155cc5a4fa49c5010b38ec4c
Successfully built cvzone


In [ ]:
import cv2
import pandas as pd
from ultralytics import YOLO
import cvzone



model=YOLO('/content/drive/MyDrive/Vehicle Crash Detection/Vehicle Dataset/runs/detect/train5/weights/best.pt')


def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :
        point = [x, y]
        print(point)



cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)





cap=cv2.VideoCapture('/content/drive/MyDrive/Vehicle Crash Detection/cr.mp4')


my_file = open("/content/drive/MyDrive/Vehicle Crash Detection/coco.txt", "r")
data = my_file.read()
class_list = data.split("\n")
#print(class_list)



count=0

while True:
    ret,frame = cap.read()
    if not ret:
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        continue


    count += 1
    if count % 3 != 0:
        continue
    frame=cv2.resize(frame,(1020,500))
    results=model.predict(frame)
 #   print(results)
    a=results[0].boxes.data
    px=pd.DataFrame(a).astype("float")


#    print(px)
    for index,row in px.iterrows():
#        print(row)

        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]

        cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),1)
        cvzone.putTextRect(frame,f'{c}',(x1,y1),1,1)


    cv2.imshow("RGB", frame)
    if cv2.waitKey(1)&0xFF==27:
        break
cap.release()
cv2.destroyAllWindows()